In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import scale
from sklearn.cross_validation import train_test_split, ShuffleSplit
from classifiers import TransparentLogisticRegression, TransparentLinearRegression
from matplotlib import pylab as pl
from scipy.sparse import diags
from IPython import display
from scale import decision_tree_scale
from IPython.display import display, HTML
from ipy_table import *

In [2]:
def transform(X, mns, sstd, axis=0):
    
    X = np.asanyarray(X)
    mns = np.asanyarray(mns)
    
    if axis and mns.ndim < X.ndim:

        return ((X - np.expand_dims(mns, axis=axis)) /
                    np.expand_dims(sstd, axis=axis))
    else:

        return (X - mns) / sstd

In [3]:
def sigmoid_afterscale(C):
    
    return 1/(1+np.exp(-C))

In [4]:
def sigmoid(x, mean, std, w, C=0):
    ex = np.exp(-((x-mean)/std*w + C))
    return 1/(1+ex)

In [5]:
def sigmoid_binary(x, mean, std, w, C=0):
    ex = np.exp(-(x*w + C))
    return 1/(1+ex)

In [6]:
#hepatitis
# WorKed
dataset = "hepatitis.csv"
class_index = 19
num_cols = 20
classes= ['DIE', 'LIVE']


#heart-c
dataset = "heart-c.csv"
class_index = 22
num_cols = 23
classes= ['<50', '>50_1']

#credit-g
dataset = "credit-g.csv"
class_index = 61
num_cols = 62
classes= ['good', 'bad']

# car
dataset = "car.csv"
class_index = 21
num_cols = 22
classes = ['acc', 'unacc']

#cmc
# WorKed
dataset = "cmc.csv"
class_index = 21
num_cols = 22
classes= ['1', '2']

#heart-statlog
# WorKed
dataset = "heart-statlog.csv"
class_index = 13
num_cols = 14
classes= ['absent', 'present']


#hepatitis
# WorKed
dataset = "hepatitis.csv"
class_index = 19
num_cols = 20
classes= ['DIE', 'LIVE']


read_cols = [i for i in range(num_cols) if i != class_index]
file_path = "D:\\IIT_Master\\2016 Spring\\CS597\\uci\\uci\\uci-tar\\nominal\\"+dataset

print dataset

hepatitis.csv


In [7]:
X = np.loadtxt(file_path, dtype=float, delimiter=",", skiprows=1, \
                   usecols=read_cols)
y = np.loadtxt(file_path, dtype=int, delimiter=",", skiprows=1, \
                   usecols=(class_index,), converters={class_index: lambda x: classes.index(x)})

num_inst, num_feat = np.shape(X)
print "The shape of this data set:",np.shape(X)

The shape of this data set: (155L, 19L)


In [8]:
# Determine binary features
num_features = X.shape[1]
non_binary = []
binary = []
for i in range(num_features):
    if len(np.unique(X[:,i])) != 2:
        non_binary.append(i)
    else:
        binary.append(i)
        
print binary

# for binary features, replace zeros with -1, assuming the other values are 1; a more correct way would check if this was true.
if len(binary) > 0:
    X_b = X[:,binary]
    X_b[X_b == 0] = -1
    X[:,binary] = X_b
    
    
if len(non_binary) == 0:
    print "123"
    raise BaseException("There are only binary features in this data set")

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 18]


In [9]:
clf_ori = TransparentLogisticRegression()
clf_ss = TransparentLogisticRegression()
clf_ig = TransparentLogisticRegression()

X_ori = X.copy()
X_ss = X.copy()
X_ig = X.copy()


if len(non_binary) > 0:
    X_ss[:,non_binary]=scale(X_ss[:,non_binary])

if len(non_binary) > 0:
    X_ig[:,non_binary]=clf_ig.fit_transform(X_ig[:,non_binary],y)
    
print binary
print non_binary

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 18]
[0, 13, 14, 15, 16, 17]


In [10]:
clf_ss.fit(X_ss, y)
clf_ig.fit(X_ig, y)

TransparentLogisticRegression(C=1.0, class_weight=None, dual=False,
               fit_intercept=True, intercept_scaling=1, max_iter=100,
               multi_class='ovr', n_jobs=1, penalty='l2',
               random_state=None, solver='liblinear', tol=0.0001,
               verbose=0, warm_start=False)

In [11]:
w1 = clf_ss.coef_[0]
w2 = clf_ig.coef_[0]

print w1
print w2

bias1 = clf_ss.intercept_
bias2 = clf_ig.intercept_

print bias1, bias2

print 1/(1+np.exp(-bias1))
print 1/(1+np.exp(-bias2))
print sum(y)/float(len(y))

[-0.45459924 -0.72519286  0.34980367 -0.32838258 -0.01333672 -0.41988771
  0.52779761 -0.06055821  0.40214723 -0.32430781 -0.78112944 -0.67204161
 -0.47718273 -0.6449203   0.08433274 -0.11577172  0.49946952  0.22956831
  0.03156807]
[-0.41296916 -0.70454171  0.32210032 -0.27632421 -0.04846058 -0.43484317
  0.51734112 -0.04482552  0.35071694 -0.33473134 -0.76490474 -0.57144876
 -0.43914814 -0.63298821  0.27957177 -0.08504893  0.53947204  0.50482749
  0.00408032]
[ 1.53229439] [ 1.45781755]
[ 0.82234176]
[ 0.81119865]
0.793548387097


In [12]:
max_list = np.max(X, axis = 0 )
min_list = np.min(X, axis = 0)

print max_list
print min_list

[  78.     1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
    1.     1.     8.   295.   648.     6.4  100.     1. ]
[  7.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.
  -1.    0.3  26.   14.    2.1   0.   -1. ]


In [13]:
mean1_ = np.zeros(shape = (1, num_feat))
mean1 = mean1_.flatten()

mean2_ = np.zeros(shape = (1, num_feat))
mean2 = mean2_.flatten()


mean1[non_binary] = np.mean(X[:,non_binary], axis = 0)
mean2[non_binary] = clf_ig.mns


std1_ = np.ones(shape = (1, num_feat))
std1 = std1_.flatten()

std2_ = np.ones(shape = (1, num_feat))
std2 = std2_.flatten()



std1[non_binary] = np.std(X[:,non_binary], axis = 0)
std2[non_binary] = clf_ig.sstd

print mean1.tolist()
print mean2.tolist()

[41.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4275167870967742, 105.32539685806452, 85.89403974193549, 3.817266167741935, 61.85227284516129, 0.0]
[29.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.35, 61.0, 47.0, 3.858633, 44.5, 0.0]


In [14]:
print non_binary

print max_list

print min_list

[0, 13, 14, 15, 16, 17]
[  78.     1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
    1.     1.     8.   295.   648.     6.4  100.     1. ]
[  7.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.
  -1.    0.3  26.   14.    2.1   0.   -1. ]


In [15]:
upper_bound = []
lower_bound = []

for i_ in range(num_feat):
    
    if i_ in non_binary:
        if mean1[i_] > mean2[i_]:
            upper_bound.append(mean1[i_])
            lower_bound.append(mean2[i_])
        else:
            upper_bound.append(mean2[i_])
            lower_bound.append(mean1[i_])
    else:
        upper_bound.append(1)
        lower_bound.append(-1)
        
print upper_bound
print lower_bound

[41.200000000000003, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.4275167870967742, 105.32539685806452, 85.894039741935487, 3.8586330000000002, 61.852272845161288, 1]
[29.0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1.3500000000000001, 61.0, 47.0, 3.8172661677419351, 44.5, -1]


In [16]:
currentX = np.copy(lower_bound)
counter = 100

for j in range(counter):
    
    for i in range(num_feat):

        if i in binary:
            num_ = 2           
           
        else:
            num_ = 10
        
        xs = np.linspace(lower_bound[i], upper_bound[i] , num = num_)
        
        currentX_ss = np.copy(currentX)
        currentX_ss[non_binary] = transform(currentX[non_binary], mean1[non_binary], std1[non_binary])
        
        currentX_ig = np.copy(currentX)
        currentX_ig[non_binary] = transform(currentX[non_binary], mean2[non_binary], std2[non_binary])
        
        C1 = np.dot(currentX_ss, w1) + bias1 - currentX_ss[i] * w1[i]
        C2 = np.dot(currentX_ig, w2) + bias2 - currentX_ig[i] * w2[i]
        
        value = abs(sigmoid(xs, mean1[i], std1[i], w1[i], C1) - sigmoid(xs, mean2[i], std2[i], w2[i], C2))
    
        currentX[i] = xs[np.argmax(value)]
        
        print np.max(value), currentX

0.00678411120135 [ 29.          -1.          -1.          -1.          -1.          -1.          -1.
  -1.          -1.          -1.          -1.          -1.          -1.
   1.35        61.          47.           3.81726617  44.5         -1.        ]
0.022784736817 [ 29.           1.          -1.          -1.          -1.          -1.          -1.
  -1.          -1.          -1.          -1.          -1.          -1.
   1.35        61.          47.           3.81726617  44.5         -1.        ]
0.022784736817 [ 29.           1.          -1.          -1.          -1.          -1.          -1.
  -1.          -1.          -1.          -1.          -1.          -1.
   1.35        61.          47.           3.81726617  44.5         -1.        ]
0.0264355435015 [ 29.           1.          -1.           1.          -1.          -1.          -1.
  -1.          -1.          -1.          -1.          -1.          -1.
   1.35        61.          47.           3.81726617  44.5         -1.       

In [17]:
print currentX

currentX_result_ss = np.copy(currentX)
currentX_result_ig = np.copy(currentX)

currentX_result_ss[non_binary] = transform(currentX[non_binary], mean1[non_binary], std1[non_binary])
currentX_result_ig[non_binary] = transform(currentX[non_binary], mean2[non_binary], std2[non_binary])

print clf_ss.predict_proba(currentX_result_ss)
print clf_ig.predict_proba(currentX_result_ig)
print clf_ss.predict_proba(currentX_result_ss) - clf_ig.predict_proba(currentX_result_ig)

[ 29.           1.           1.          -1.           1.           1.          -1.
  -1.          -1.           1.           1.          -1.          -1.
   1.42751679  61.          47.           3.81726617  44.5          1.        ]
[[ 0.3920676  0.6079324]]
[[ 0.52930416  0.47069584]]
[[-0.13723655  0.13723655]]


C:\Python27\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Python27\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Python27\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Python27\lib\site-packages\sklearn

In [18]:
print currentX.tolist()

[29.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.4275167870967742, 61.0, 47.0, 3.817266167741935, 44.5, 1.0]
